In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv("DF_modelo.csv", index_col=0)
df_train

,ID,Inicio Temporada,EDAD,FEC_ALTA,% de asistencia,CUOTA_PAGADA,NOMBRE_SECTOR,Renuevan / No Renuevan
0,7b259c2e043143b,2015,85,9,20.689655,748.00,14,0
1,0f8a86a92fc222d,2015,32,0,44.827586,935.00,14,1
2,3dabbd4e5974ea9,2015,83,0,65.517241,467.00,0,1
3,df72af73ff33004,2015,54,0,68.965517,519.00,0,1
4,e660f665850d2b5,2015,61,0,72.413793,408.00,9,1
...,...,...,...,...,...,...,...,...
155287,036ef2caef5e700,2021,55,11,94.117647,715.62,14,1
155288,73738fb86ca255a,2021,30,14,82.352941,631.43,14,1
155289,2e21500212c12bf,2021,79,14,82.352941,568.29,14,1
155290,caf066457e10021,2021,86,9,94.117647,568.29,14,1


In [4]:
df_train.drop(columns=["ID"], inplace=True)

In [5]:
df_train[df_train["Inicio Temporada"]<2020]

,Inicio Temporada,EDAD,FEC_ALTA,% de asistencia,CUOTA_PAGADA,NOMBRE_SECTOR,Renuevan / No Renuevan
0,2015,85,9,20.689655,748.0,14,0
1,2015,32,0,44.827586,935.0,14,1
2,2015,83,0,65.517241,467.0,0,1
3,2015,54,0,68.965517,519.0,0,1
4,2015,61,0,72.413793,408.0,9,1
...,...,...,...,...,...,...,...
133161,2019,55,11,100.000000,1009.0,14,1
133162,2019,30,14,94.117647,1121.0,14,1
133163,2019,56,9,94.117647,831.7,14,0
133164,2019,86,9,94.117647,1009.0,14,1


In [6]:
x_train = df_train[df_train["Inicio Temporada"]<2020].drop(columns=["Renuevan / No Renuevan", "Inicio Temporada"])
y_train = df_train[df_train["Inicio Temporada"]<2020].iloc[:,-1]
x_test = df_train[df_train["Inicio Temporada"]==2021].drop(columns=["Renuevan / No Renuevan", "Inicio Temporada"])
y_test = df_train[df_train["Inicio Temporada"]==2021].iloc[:,-1]

### XG BOOST CLASSIFIER

In [49]:
import xgboost
xgb = xgboost.XGBClassifier(max_depth = 4, n_estimators = 80, n_jobs = -1, early_stopping_rounds = 8)
xgb.fit(x_train,y_train, eval_set = [(x_test, y_test)])

[0]	validation_0-logloss:0.54179
[1]	validation_0-logloss:0.45596
[2]	validation_0-logloss:0.40248
[3]	validation_0-logloss:0.36682
[4]	validation_0-logloss:0.34463
[5]	validation_0-logloss:0.32919
[6]	validation_0-logloss:0.32006
[7]	validation_0-logloss:0.31327
[8]	validation_0-logloss:0.31184
[9]	validation_0-logloss:0.30693
[10]	validation_0-logloss:0.31170
[11]	validation_0-logloss:0.30880
[12]	validation_0-logloss:0.30725
[13]	validation_0-logloss:0.30582
[14]	validation_0-logloss:0.31105
[15]	validation_0-logloss:0.30983
[16]	validation_0-logloss:0.31188
[17]	validation_0-logloss:0.31212
[18]	validation_0-logloss:0.31563
[19]	validation_0-logloss:0.31530
[20]	validation_0-logloss:0.31600
[21]	validation_0-logloss:0.31596


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=8,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=80, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [10]:
from sklearn.metrics import accuracy_score

In [50]:
pred_val = xgb.predict(x_test)
pred_train = xgb.predict(x_train)

print('Accuracy en train:',xgb.score(x_train, y_train))
print('Accuracy en val:',accuracy_score(y_test, pred_val))

Accuracy en train: 0.845733510962585
Accuracy en val: 0.901877210483359


In [51]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred_val)


array([[  129,  1743],
       [  421, 19761]], dtype=int64)

In [43]:
df_testeo = pd.read_csv('DF_modelo_test.csv').iloc[:,3:]
df_testeo

,EDAD,FEC_ALTA,% de asistencia,CUOTA_PAGADA,NOMBRE_SECTOR
0,34,16,100.000000,901.0,12
1,65,7,92.857143,811.0,12
2,45,9,78.571429,859.0,12
3,43,9,100.000000,750.0,6
4,48,9,100.000000,750.0,6
...,...,...,...,...,...
26586,58,18,100.000000,1009.0,12
26587,66,16,85.714286,1121.0,12
26588,79,14,92.857143,1009.0,12
26589,23,12,100.000000,908.0,12


In [48]:
pred_testeo = xgb.predict(df_testeo)
pred_testeo.sum() / len(pred_testeo)

0.961979617163702